In [3]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pickle
import pandas as pd 
import math
import numpy as np
import torch

from bayes_opt.utils import test_learning
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from Models.cslr import cslr

train_sets = pickle.load(open("datasets/train_sets_bank.dat","rb"))
test_sets = pickle.load(open("datasets/test_sets_bank.dat","rb"))
val_sets = [pickle.load(open("datasets/val_sets_bank.dat","rb")), pickle.load(open("datasets/val_test_sets_bank.dat","rb"))]
features = pickle.load(open("datasets/bank_features","rb"))
returns = pickle.load(open("datasets/bank_returns","rb"))
outcomes = pickle.load(open("datasets/bank_outcomes","rb"))
rmax = np.zeros(len(returns))
for i in range(len(returns)):
    rmax[i] = returns[i,np.argmax(returns[i])]
    
replication = 5
n_splits = 10
n_trials = 50
n_epochs = 1000
n_steps = 1000
device = torch.device("cpu")
bank_results = []
max_batchsize = math.floor(math.log2(features.shape[0]))
batch_norm = [False]
lr_rates = [0.00005, 0.01]
batch_sizes = [8, max_batchsize]
bayes = True
time_limit = 600
scaler = MinMaxScaler()

for bn in batch_norm:

    for rep in range(replication):

        results = (np.zeros((n_splits,12)) if bn is False else np.zeros((n_splits,4)))
                   
        for i in range(n_splits):

            config = {
                "rep": rep,
                "set": i,
                "bayes": bayes,
                "n_trials": n_trials,
                "max_batch": max_batchsize,
                "lr": lr_rates,
                "batches": batch_sizes,
                "val_sets": val_sets,
                "n_epochs": n_epochs,
                "n_steps": n_steps,
                "batchnorm": bn,
                "device": device,
                "scaler": scaler,
                "time_limit": time_limit
            }
            
            x_train, x_test = features[train_sets[rep][i]], features[test_sets[rep][i]]
            r_train, r_test = returns[train_sets[rep][i]], returns[test_sets[rep][i]]
            rmax_train, rmax_test = rmax[train_sets[rep][i]], rmax[test_sets[rep][i]]           
            model = cslr(config,
                         x_train,
                         r_train,
                         x_test,
                         r_test)
            
            test_probs, train_probs = model.gradient()
            test_return, test_outcome = test_learning(test_probs, r_test)
            train_return, _ = test_learning(train_probs, r_train)
            results[i,0], results[i,1] = test_return/sum(rmax_test), train_return/sum(rmax_train)
            results[i,2] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
            results[i,3] = f1_score(outcomes[test_sets[rep][i]], test_outcome)       
            
            if bn is False:
                
                model.give_initial = True
                test_probs, train_probs = model.mip_opt()
                test_return, test_outcome = test_learning(test_probs, r_test)
                train_return, _ = test_learning(train_probs, r_train)
                results[i,4], results[i,5] = test_return/sum(rmax_test), train_return/sum(rmax_train)
                results[i,6] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
                results[i,7] = f1_score(outcomes[test_sets[rep][i]], test_outcome)    

                model.give_initial = False
                test_probs, train_probs = model.mip_opt()
                test_return, test_outcome = test_learning(test_probs, r_test)
                train_return, _ = test_learning(train_probs, r_train)
                results[i,8], results[i,9] = test_return/sum(rmax_test), train_return/sum(rmax_train)
                results[i,10] = accuracy_score(outcomes[test_sets[rep][i]], test_outcome)
                results[i,11] = f1_score(outcomes[test_sets[rep][i]], test_outcome)                
        print("end of set")
        print("end of set")    
        print("end of set")    
        print("end of set")    
        print("end of set")            
        bank_results.append(results)    
        pickle.dump(bank_results, open("bank_results_minmax_last", "wb"))         
        

Epoch 1000 Train Return: 115050.391.  Validation Return: 33098.023. Elapsed time: 6.720s.
Epoch 1000 Train Return: 115958.586.  Validation Return: 6457.358. Elapsed time: 6.800s.
Epoch 1000 Train Return: 115978.461.  Validation Return: 6437.467. Elapsed time: 6.736s.
Epoch 1000 Train Return: 91844.008.  Validation Return: 30571.924. Elapsed time: 6.770s.
Epoch 1000 Train Return: 103049.141.  Validation Return: 19366.781. Elapsed time: 6.826s.
Epoch 1000 Train Return: 142366.188.  Validation Return: -7583.483. Elapsed time: 6.740s.
Epoch 1000 Train Return: 106673.836.  Validation Return: 15742.084. Elapsed time: 6.790s.
Epoch 1000 Train Return: 113974.117.  Validation Return: 8441.820. Elapsed time: 6.447s.
Epoch 1000 Train Return: 118359.258.  Validation Return: 4056.661. Elapsed time: 6.516s.
Epoch 1000 Train Return: 112118.219.  Validation Return: 10297.710. Elapsed time: 6.530s.


[I 2020-10-15 11:07:28,656] Finished trial#0 with value: 12688.634482073783 with parameters: {'lr_rate': 0.007507555424142269, 'batch_size': 8}. Best is trial#0 with value: 12688.634482073783.


Epoch 1000 Train Return: -93788.328.  Validation Return: -28627.605. Elapsed time: 2.779s.
Epoch 1000 Train Return: 152344.969.  Validation Return: 6457.358. Elapsed time: 2.619s.
Epoch 1000 Train Return: 128645.438.  Validation Return: 6358.890. Elapsed time: 2.695s.
Epoch 1000 Train Return: 104431.133.  Validation Return: 30262.156. Elapsed time: 2.698s.
Epoch 1000 Train Return: 153864.516.  Validation Return: 18122.273. Elapsed time: 2.677s.
Epoch 1000 Train Return: 163136.422.  Validation Return: -8324.406. Elapsed time: 2.560s.
Epoch 1000 Train Return: 241649.438.  Validation Return: 13471.059. Elapsed time: 2.757s.
Epoch 1000 Train Return: 113974.117.  Validation Return: 8441.820. Elapsed time: 2.664s.
Epoch 1000 Train Return: 267373.312.  Validation Return: 3044.674. Elapsed time: 2.613s.
Epoch 1000 Train Return: 112118.219.  Validation Return: 10297.710. Elapsed time: 2.770s.


[I 2020-10-15 11:07:55,814] Finished trial#1 with value: 5943.420724368096 with parameters: {'lr_rate': 0.003608793055597558, 'batch_size': 11}. Best is trial#0 with value: 12688.634482073783.


Epoch 1000 Train Return: 93788.328.  Validation Return: 28627.605. Elapsed time: 6.218s.
Epoch 1000 Train Return: 212940.344.  Validation Return: 15830.364. Elapsed time: 6.380s.
Epoch 1000 Train Return: 115978.461.  Validation Return: 6437.467. Elapsed time: 6.322s.
Epoch 1000 Train Return: 91844.008.  Validation Return: 30571.924. Elapsed time: 6.373s.
Epoch 1000 Train Return: 103049.141.  Validation Return: 19366.781. Elapsed time: 6.425s.
Epoch 1000 Train Return: 129999.414.  Validation Return: -7583.483. Elapsed time: 6.370s.
Epoch 1000 Train Return: -106330.164.  Validation Return: -15742.084. Elapsed time: 6.433s.
Epoch 1000 Train Return: 113974.117.  Validation Return: 8441.820. Elapsed time: 6.341s.
Epoch 1000 Train Return: 128899.188.  Validation Return: 6322.400. Elapsed time: 6.504s.
Epoch 1000 Train Return: 112118.219.  Validation Return: 10297.710. Elapsed time: 6.363s.


...

Epoch 1000 Train Return: 196432.781.  Validation Return: 23396.627. Elapsed time: 4.913s.
Epoch 1000 Train Return: 229433.266.  Validation Return: 4290.606. Elapsed time: 4.870s.
Epoch 1000 Train Return: 316498.719.  Validation Return: 22986.980. Elapsed time: 5.108s.
Epoch 1000 Train Return: 312698.531.  Validation Return: 16387.934. Elapsed time: 4.845s.
Epoch 1000 Train Return: 172376.469.  Validation Return: 22157.027. Elapsed time: 5.162s.
Epoch 1000 Train Return: 150684.797.  Validation Return: -6779.695. Elapsed time: 4.969s.
Epoch 1000 Train Return: 206501.062.  Validation Return: 27076.480. Elapsed time: 4.830s.
Epoch 1000 Train Return: 139966.062.  Validation Return: 111.301. Elapsed time: 5.097s.
Epoch 1000 Train Return: 352228.469.  Validation Return: 23125.932. Elapsed time: 4.759s.
Epoch 1000 Train Return: 389238.938.  Validation Return: 20626.564. Elapsed time: 4.774s.


[I 2020-10-17 04:08:40,527] Finished trial#48 with value: 15587.56157515049 with parameters: {'lr_rate': 0.0004255254700650784, 'batch_size': 9}. Best is trial#16 with value: 20877.64143908024.


Epoch 1000 Train Return: 128052.320.  Validation Return: 16485.666. Elapsed time: 6.306s.
Epoch 1000 Train Return: 142528.000.  Validation Return: -1847.303. Elapsed time: 6.175s.
Epoch 1000 Train Return: 357804.094.  Validation Return: 34401.770. Elapsed time: 6.550s.
Epoch 1000 Train Return: 346425.625.  Validation Return: 13948.548. Elapsed time: 6.447s.
Epoch 1000 Train Return: 174769.641.  Validation Return: 22738.488. Elapsed time: 6.283s.
Epoch 1000 Train Return: 292184.812.  Validation Return: 10807.146. Elapsed time: 6.582s.
Epoch 1000 Train Return: 254367.219.  Validation Return: 33013.430. Elapsed time: 6.393s.
Epoch 1000 Train Return: 136724.828.  Validation Return: 111.301. Elapsed time: 6.174s.
Epoch 1000 Train Return: 125293.836.  Validation Return: 11542.290. Elapsed time: 6.549s.
Epoch 1000 Train Return: 101316.797.  Validation Return: 35519.340. Elapsed time: 6.580s.


[I 2020-10-17 04:09:44,927] Finished trial#49 with value: 17605.974550700186 with parameters: {'lr_rate': 0.0023692564194802484, 'batch_size': 8}. Best is trial#16 with value: 20877.64143908024.


Epoch 1000 Train Return: 153656.188.  Validation Return: -1489.671. Elapsed time: 5.452s.
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 18432 rows, 13946 columns and 225078 nonzeros
Variable types: 4730 continuous, 9216 integer (9216 binary)
Coefficient statistics:
  Matrix range     [9e-06, 2e+02]
  Objective range  [1e+00, 6e+03]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e-02, 1e+00]

MIP start produced solution with objective 153182 (0.03s)
Loaded MIP start with objective 153182

Presolve removed 9218 rows and 6918 columns
Presolve time: 0.57s
Presolved: 9214 rows, 7028 columns, 199026 nonzeros
Variable types: 4724 continuous, 2304 integer (2304 binary)

Root relaxation: objective 6.489861e+05, 3962 iterations, 2.08 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 648986.

In [2]:
np.mean(np.mean(bank_results,axis=0),axis=0)

array([0.32596522, 0.44027375, 0.70257296, 0.30352698, 0.32240163,
       0.5404884 , 0.71858706, 0.31025852, 0.14578763, 0.37868664,
       0.63810402, 0.18782811])

In [4]:
np.mean(np.mean(bank_results,axis=0),axis=0)

array([0.26568596, 0.36897917, 0.84759363, 0.17895576, 0.32402055,
       0.61561012, 0.83857004, 0.28301588])

In [6]:
np.sum(returns, axis=0)/sum(rmax)

array([ 0.18517232, -0.18517232])

In [5]:
pickle.dump(bank_results, open("bank_results_bn_std", "wb"))